# Projet de Maths 1

## Contexte

On se donne une fonction $f: \mathbb{R}^2 \to \mathbb{R}$ continument différentiable. Notre objectif est de créer une fonction qui permet de tracer un contour pour une valeur $c$ donnée.

## Amorce 

On suppose ici $f$ définie sur $[0,1]^2$. On veut tout d'abord trouver un point de départ sur le coté gauche du carré pour tracer notre contour. Pour cela, on cherche un $y \in [0,1]$ tel que $f(0,y)=c$. 
On procède par dichotomie et l'on impose donc à $f$ de vérifier : $c \in [f(0,0),f(0,1)] $ ou $c \in [f(0,1),f(0,0)]$ .

### Principe de la dichotomie

Puisque f est continuement différentiable, elle est en particulier continue. Avec la condition que l'on impose, on peut donc utiliser le théorème des valeurs intermédiaires sur $g$ définie par : $g : x \in [0,1] \mapsto f(0,x)$. On en déduit qu'il existe $t$ dans $[0,1]$ tel que $g(t)=c$. On a donc : $t \in [0, \frac{1}{2}]$ ou $t \in [\frac{1}{2},1]$. Puis on réitère le raisonnement sur le segement auquel appartient $t$. Puisqu'on diminue par 2 la longueur du segment à chaque étape, on va converger vers un point.

In [ ]:
def find_seed(g,c=0,debut=0,fin=1,eps=2**(-26)):
    if g(debut)<=c<=g(fin) or g(fin)<=c<=g(debut):  #On vérifie que la condition est vérifiée.
        a=debut
        b=fin
    else :
        return None
    while abs(g((a+b)/2)-c)>eps:    #On choisit comme condition d'arrêt que l'image par g de la valeur que l'on renvoie soit espacée d'au plus epsilon de c.
        milieu=(a+b)/2              
        if g(a)<=c<=g(milieu) or g(a)>=c>=g(milieu):    # On cherche si t est dans [a, milieu] ou pas 
            b=milieu
        else :
            a=milieu
    return float((a+b)/2)

### Exemple

0n teste notre fonction sur $f : x \mapsto x^2 -2$. On va obtenir une valeur approchée à eps de $\sqrt{2}$.

In [ ]:
def f_1(x):
    return x**2 -2

find_seed(f_1,0,1,2)

## Contour simple 

On peut maintenant, à partir de la fonction précédente qui nous donne un point de départ, trouver un fragment de ligne de niveau.
On procède de la manière suivante : à partir du point que l'on vient de trouver, on calcule le gradient de $f$ en ce point puis un vecteur normal au gradient de norme $\delta$, puis on se dirige dans la direction du vecteur normal au gradient (ou dans la direction opposée si l'on sort du carré). Ainsi, on obtient une liste de points espacés de $\delta$ qui sont censés vérifier $f(x,y)=c$.

In [ ]:
import autograd
from autograd import numpy as np
import matplotlib.pyplot as plt

def simple_contour(f,c=0.0,delta=0.01,nb_point=100):

    def grad_f(x,y):                          # calcule le gradient de f à partir de autograd
        gr=autograd.grad
        return np.r_[gr(f,0)(x,y),gr(f,1)(x,y)]

    def g(t):                                # pour calculer le point de départ du contour sur le coté gauche
        return f(0,t)

    x=[0.0]                                    # point de départ
    y=[find_seed(g,c)]

    if y[0]==None:                   #si l'on n'a pas trouvé de point de départ
        return [],[]

    for i in range(nb_point):                 # on limte le nombre de points que l'on cherche pour avoir un programme qui finit
        grad=grad_f(x[i],y[i])                # on calcule le gradient au dernier point que l'on a trouvé
        aux=[grad[1],-grad[0]]                # puis un vecteur auxiliaire normal au gradient
        norme_aux=np.sqrt(aux[0]**2+aux[1]**2)  # puis sa norme pour le renormaliser
        vect_orthon=[delta/norme_aux*aux[0],delta/norme_aux*aux[1]]   # on calcule un vecteur normal au gradient de norme delta

        posx=x[i]+vect_orthon[0]      # abscisse du potentiel point suivant
        posy=y[i]+vect_orthon[1]      # ordonnée du potentiel point suivant


        if posx>1 or posx<0 or posy>1 or posy<0: # si jamais on sort du carré, on part dans l'autre direction
            posx=posx-2*vect_orthon[0]
            posy=posy-2*vect_orthon[1]
            
            if posx>1 or posx<0 or posy>1 or posy<0:  # si l'on sort encore, on s'arrête
                return x,y
       
        x.append(posx)    # on ajoute le nouveau point à la liste
        y.append(posy)
    
    return x,y    #on retourne le fragment de contour


### Test 

In [ ]:
def f_test(x,y):
    return np.power(x,2)+np.power(y,2)

In [ ]:
# Regardons l'aspect de la fonction 
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

domain_x=np.linspace(-10,10,100)
domain_y=np.linspace(-10,10,100)

X,Y=np.meshgrid(domain_x, domain_y)      # Création de la grille

Z=f_test(X,Y)                            # création des valeurs 

fig=plt.figure()
ax=Axes3D(fig)
ax.plot_surface(X, Y, Z)                 # Tracé de la fonction

In [ ]:
# Puis essayons de tracer une ligne de niveau

plt.close()
x,y=simple_contour(f_test,0.01)
plt.plot(x,y)
plt.show()

On contaste que le programme ne renvoie pas le résultat attendu ce qui est dû au fait que le vecteur noramal au gradient donne une direction dans laquelle il peut y avoir un point sur le contour mais c'est assez approximatif.
Pour ressoudre ce problème, on a essayé de calculer les points suivants en mettant une bouce while avec la condition d'arrêt : $\mid f(x,y)-c \mid \leq \frac{\mid c \mid}{100}$ pour ne pas trop s'éloigner de la ligne de niveau mais nous obtenions pas de résultats satisfaisants non plus car le programme n'effectuait qu'une dizaine d'étape dans le meilleur des cas.

Nous avons donc décider de changer de méthode et d'utiliser find_seed à chaque étape sur un segment du carré donné par le gradient. Pour être plus précis, on prend le point potentiel suivant en utilisant un vecteur orthogonal au gradient (on le choisit de sorte que l'on "avance" en calculant un certain produit scalaire) puis on cherche sur le segement du carré passant par ce point et parallèle au gradient pour $x$ dans $[posx-\delta,posx + \delta]$. Ainsi le point suivant $(x,y)$ vérifie $f(x,y)=c$ (à epsilon près) et il est éloigné d'approximativement $\delta$ du point d'avant. 

In [ ]:
def simple_contour(f,c=0.0,delta=0.01):

    def grad_f(x,y):
        gr=autograd.grad
        return [gr(f,0)(x,y),gr(f,1)(x,y)]

    def g(t):
        return f(0,t)

    x=[0.0]
    y=[find_seed(g,c)]

    i=0
    
    while True:            # On boucle tant qu'on ne sort pas du carré
        if x[-1]==None or y[-1]==None:
            return x[:-1],y[:-1]
        grad=grad_f(x[-1],y[-1])
        aux=[grad[1],-grad[0]]
        norme_aux=np.sqrt(aux[0]**2+aux[1]**2)
        vect_orthon=[delta/norme_aux*aux[0],delta/norme_aux*aux[1]]     #direction normale au gradient
        posx=x[-1]+vect_orthon[0]       #nouveau point autour duquel cherché
        posy=y[-1]+vect_orthon[1]

        if i==0:
            if posx>1 or posx<0 or posy>1 or posy<0:    #si on est sorti du carré, on change de direction : on part dans l'autre direction pour                                                             #revenir dans le carré
                posx=posx-2*vect_orthon[0]
                posy=posy-2*vect_orthon[1]
                if posx>1 or posx<0 or posy>1 or posy<0:    #si on sort encore du carré, le fragment est fini
                    return x,y

        elif i>=1 :
            if vect_orthon[0]*(x[i]-x[i-1])+vect_orthon[1]*(y[i]-y[i-1])<0:    #produit scalaire du gradient et de l'écart entre les positions                                                                                     #négatif --> il faut changer de direction pour ne pas revenir en                                                                                    #arrière
                posx=posx-2*vect_orthon[0]
                posy=posy-2*vect_orthon[1]
            if posx>1 or posx<0 or posy>1 or posy<0:                           #On sort encore du carré, le fragment est fini
                return x,y

        i+=1

        if grad[0]==0.0:            #Pour une direction verticale
            x.append(posx)
            y.append(find_seed(lambda t:f(posx,t),c,max(posy-delta,0),min(posy+delta,1)))  
            #on fait une dichotomie sur la droite orhtogonale au gradient passant par le point déterminé précédemment, on cherche un point distant              #de plus ou moins delta de la position précédente.

        else :
            t=find_seed(lambda t:f(t,posy+(t-posx)*grad[1]/grad[0]),c,max(posx-delta,0),min(posx+delta,1)) 
            #idem : dichotomie sur la droite orthogonale au gradient                                                                                                                                           
            x.append(t)
            try :
                y.append(posy+(t-posx)*grad[1]/grad[0]) 
            except:
                y.append(0)
    return x,y

### Exemple

In [ ]:
plt.close()
x,y=simple_contour(f_test,0.01)
plt.plot(x,y)
plt.show()

Résultat beaucoup plus satisfaisant !!

## Contour complexe

On cherche maintenant à tracer le contour d'une fonction dans un cadre général c'est-à-dire pour un domaine quelconque. Pour cela, on découpe le domaine en cellules carrées (grâce à xc et yc). On peut alors exploiter la méthode utilisée pour le contour simple sur chacune de ces cellules en renormalisant le domaine et en cherchant cette fois des amorces sur toute la frontière de la cellule.
On récupère alors un ensemble de fragments de contour qui permettent finalement de réaliser le tracé de contour voulu.

Tout d'abord, afin d'effectuer une recherche d'amorce sur tous les bords de la cellule nous avons besoin de réaliser des rotations sur la fonction initiale: nous implémentons donc une fonction rotation.

In [ ]:
def rotation(g):      #Si g est définie sur [0,1]x[0,1], renvoie la fonction g rotationnée de pi/2 de centre de rotation [1/2,1/2]
    return lambda x,y : g(y,1-x)

Nous pouvons maintenant implémenter la fonction qui va nous renvoyer les fragments du contour voulu.

In [ ]:
def contour(f,c=0.0,xc=[0.0,1.0], yc=[0.0,1.0], delta=0.01):

    xs=[]                         # Initialisation des listes qui vont contenir les abcisses et les ordonnées de la ligne de niveau
    ys=[]
    for i in range(len(xc)-1):
        for j in range(len(yc)-1):    # On parcourt les rectangles du quadrillage
            deb_x=xc[i]               # On a besoin des coordonnées des sommets du rectangle dans lequel on veut tracer un fragment de ligne de niveau pour se ramener à une fonction définie dans [0,1]x[0,1]
            fin_x=xc[i+1]
            deb_y=yc[j]
            fin_y=yc[j+1]
            ecart_x=fin_x-deb_x
            ecart_y=fin_y-deb_y

            def f_nor(x,y):                # On se ramène à une fonction 'normalisée' définie sur [0,1]x[0,1]
                return f(x*ecart_x+deb_x,y*ecart_y+deb_y)

            def ajout_renormalisation(liste_x,liste_y):    # On définit une fonction qui nous permettra de 'renormaliser' les points obtenus pour f_nor pour qu'ils correspondent à f_nor
                for x in liste_x:
                    xs.append(x*ecart_x+deb_x)
                for y in liste_y:
                    ys.append(y*ecart_y+deb_y)

            x_gauche,y_gauche=simple_contour(f_nor,c,delta)                 # On cherche un fragment de ligne de niveau en partant du coté gauche du carré [0,1]x[0,1]
            x_haut,y_haut=simple_contour(rotation(f_nor),c,delta)  # Idem en partant du haut
            x_droite,y_droite=simple_contour(rotation(rotation(f_nor)),c,delta)   # Puis de le droite 
            x_bas,y_bas=simple_contour(rotation(rotation(rotation(f_nor))),c,delta)  #Et enfin du bas 
 
            ajout_renormalisation(x_gauche,y_gauche)
            ajout_renormalisation(y_haut,[1-x for x in x_haut])                             # Puis on retourne les valeurs trouvées pour qu'elles 
            ajout_renormalisation([1-x for x in x_droite],[1-y for y in y_droite])          # correspondent bien à f_nor.
            ajout_renormalisation([1-y for y in y_bas],x_bas)

    return xs,ys   # On retourne le résultat

### Test

On réalise le test de ce programme avec la fonction $f(x,y)=2(g(x,y)-h(x,y))$ où $g(x,y)=\exp(-x^2-y^2)$ et $h(x,y)=\exp(-(x-1)^2-(y-1)^2)$.

In [ ]:
def g(x,y):
    return np.exp(-np.power(x,2)-np.power(y,2))
def h(x,y):
    return g(x-1,y-1)

def f_test2(x,y):
    return 2*(g(x,y)-h(x,y))

In [ ]:
# Aspect de cette nouvelle fonction

domain_x=np.linspace(-3,3,100)
domain_y=np.linspace(-3,3,100)

X,Y=np.meshgrid(domain_x, domain_y)      # Création de la grille

Z=f_test2(X,Y)                            # création des valeurs 

fig=plt.figure()
ax=Axes3D(fig)
ax.plot_surface(X, Y, Z)                 # Tracé de la fonction

Finalement on peut tracer les lignes de niveau de la fonction test.

In [ ]:
from scipy import*
quadri=linspace(-2,3,10).tolist()
plt.close()
for n in [-1.5,-1.0,-0.5,0.0,0.5,1.0,1.5]:
    x,y=contour(f_test2,n,quadri,quadri)
    plt.scatter(x,y,s=1)
plt.xlim(-2.0,3.0)
plt.ylim(-2.0,3.0)
plt.show()

## Conclusion

Améliorations possibles : 

Dans notre version du programme, les points ne sont pas vraiment espacés de delta. On pourrait modifier notre programme pour qu'il fasse une dichotomie sur un cercle de rayon delta et centré sur le point que l'on vient de trouver pour qu'on ait les points espacés exactement de delta.


On pourrait aussi utiliser une méthode de Newton plutôt qu'une dichotomie ce qui convergerait plus vite.